# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import imblearn
from imblearn.over_sampling import SMOTE
   

In [2]:
# Your code here
fraud_df = pd.read_csv('fraud.csv').sample(100000)
fraud_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
5073566,355,CASH_OUT,158185.91,C115070170,262.00,0.00,C577333345,350766.76,508952.67,0,0
5563268,386,CASH_OUT,9723.82,C1593579457,98225.96,88502.14,C44247024,3963698.23,3973422.05,0,0
496429,20,CASH_IN,28506.65,C230509228,585295.70,613802.34,C368761144,2245989.48,2113160.89,0,0
44758,9,PAYMENT,3326.12,C1591821227,76474.69,73148.57,M490347577,0.00,0.00,0,0
4962368,351,CASH_OUT,50398.68,C735380759,0.00,0.00,C932597815,176759.01,227157.69,0,0


### What is the distribution of the outcome? 

In [3]:
# Your response here
fraud_df["isFraud"].value_counts()

0    99884
1      116
Name: isFraud, dtype: int64

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [4]:
# Your code here
fraud_df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [5]:
fraud_df["type"].value_counts()

CASH_OUT    35098
PAYMENT     33775
CASH_IN     22149
TRANSFER     8309
DEBIT         669
Name: type, dtype: int64

In [6]:
fraud_df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [7]:
fraud_df=pd.get_dummies(fraud_df, columns=['type'])
fraud_df.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
5073566,355,158185.91,C115070170,262.00,0.00,C577333345,350766.76,508952.67,0,0,0,1,0,0,0
5563268,386,9723.82,C1593579457,98225.96,88502.14,C44247024,3963698.23,3973422.05,0,0,0,1,0,0,0
496429,20,28506.65,C230509228,585295.70,613802.34,C368761144,2245989.48,2113160.89,0,0,1,0,0,0,0
44758,9,3326.12,C1591821227,76474.69,73148.57,M490347577,0.00,0.00,0,0,0,0,0,1,0
4962368,351,50398.68,C735380759,0.00,0.00,C932597815,176759.01,227157.69,0,0,0,1,0,0,0


### Run a logisitc regression classifier and evaluate its accuracy.

In [14]:
# Your code here
X = fraud_df.drop(['nameOrig','nameDest', 'isFraud'],axis=1)
y = fraud_df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test, y_test)
pred = model.predict(X_test)


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
print(classification_report(y_test, pred))
confusion_matrix(y_test,pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     24969
           1       0.45      0.48      0.47        31

    accuracy                           1.00     25000
   macro avg       0.73      0.74      0.73     25000
weighted avg       1.00      1.00      1.00     25000



array([[24951,    18],
       [   16,    15]])

### Now pick a model of your choice and evaluate its accuracy.

In [17]:
# Your code here
sm = SMOTE(random_state=1, sampling_strategy=0.05) 
X_train_smote, y_train_smote = sm.fit_resample(X_train,y_train)
model_smote= LogisticRegression(max_iter=1000)
model_smote.fit(X_train_smote,y_train_smote)
pred = model_smote.predict(X_test)
print(classification_report(y_test, pred))
confusion_matrix(y_test,pred)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     24969
           1       0.09      0.68      0.16        31

    accuracy                           0.99     25000
   macro avg       0.55      0.83      0.58     25000
weighted avg       1.00      0.99      0.99     25000



array([[24762,   207],
       [   10,    21]])

### Which model worked better and how do you know?

In [10]:
# Your response here
#recall is higher now for case 1

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.